In [ ]:
import sys, importlib
sys.path.append(r"C:/Users/alber/OneDrive/Desktop/Proyectos/Baratazo")  # usa r"..." en Windows
import os
import re
import math
from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import IntegrityError

# Scrapers
from mercadona import scrape_mercadona
from guardar_mercadona import reload_mercadona

from bonpreu import scrape_bonpreu
from consum import scrape_consum


In [ ]:
def _num_es_txt(s: str):
    if not s:
        return None
    s = str(s).replace("\xa0", " ").strip()
    m = re.search(r"(\d{1,3}(?:\.\d{3})*(?:,\d+)|\d+,\d+|\d+(?:\.\d+)?)", s)
    if not m:
        return None
    t = m.group(1)
    if "," in t and "." in t:
        t = t.replace(".", "").replace(",", ".")
    elif "," in t:
        t = t.replace(",", ".")
    try:
        return float(t)
    except:
        return None


def _ppu_simple(ppu_text):
    """
    Extrae el número que aparece antes del símbolo € en price_per_unit_text.
    Ejemplos:
      '(1,19 € per article)' -> 1.19
      '0,75 € / 100 g' -> 0.75
      '2.50 €/L' -> 2.50
    Devuelve float o None.
    """
    if not ppu_text:
        return None
    s = str(ppu_text)
    m = re.search(r"([\d.,]+)\s*€", s)
    if not m:
        return None
    t = m.group(1).replace(".", "").replace(",", ".")
    try:
        return float(t)
    except:
        return None


def _extract_qty(text: str):
    """
    Devuelve (total_kg, total_l) a partir de cadenas como:
    'Garrafa 5 L', 'Botella 1,5 L', '500 g', 'Pack 6 x 330 ml', '2x1 L', '6×33 cl'...
    """
    if not text:
        return (None, None)
    t = text.lower().replace(",", ".")
    # pack: NxY(unidad)
    m = re.search(r"(\d+)\s*[x×]\s*(\d+(?:\.\d+)?)\s*(kg|g|l|ml|cl)", t)
    if m:
        n = int(m.group(1))
        q = float(m.group(2))
        u = m.group(3)
        if u == "kg":
            return (n * q, None)
        if u == "g":
            return (n * (q / 1000.0), None)
        if u == "l":
            return (None, n * q)
        if u == "ml":
            return (None, n * (q / 1000.0))
        if u == "cl":
            return (None, n * (q / 100.0))
    # simple: Y(unidad)
    m = re.search(r"(\d+(?:\.\d+)?)\s*(kg|g|l|ml|cl)", t)
    if m:
        q = float(m.group(1))
        u = m.group(2)
        if u == "kg":
            return (q, None)
        if u == "g":
            return (q / 1000.0, None)
        if u == "l":
            return (None, q)
        if u == "ml":
            return (None, q / 1000.0)
        if u == "cl":
            return (None, q / 100.0)
    return (None, None)


def _compute_row(r: pd.Series):
    ppt = (r.get("price_per_unit_text") or "").lower()
    price = r.get("price")
    ppk = ppl = None

    # 1) Si el propio sitio ya da €/kg o €/l, úsalo
    if "€/kg" in ppt or "€/ kilo" in ppt:
        ppk = _num_es_txt(ppt)
    if "€/l" in ppt or "€/litro" in ppt:
        ppl = _num_es_txt(ppt)

    # 2) Si no viene, infiere desde el formato/nombre
    if (ppk is None and ppl is None) and price is not None and not (isinstance(price, float) and math.isnan(price)):
        txt = f"{r.get('format_text','')} {r.get('name','')}"
        total_kg, total_l = _extract_qty(txt)
        if total_kg and total_kg > 0:
            ppk = float(price) / total_kg
        if total_l and total_l > 0:
            ppl = float(price) / total_l

    # Redondeos
    if ppk is not None:
        ppk = round(ppk, 4)
    if ppl is not None:
        ppl = round(ppl, 4)
    return pd.Series({"price_per_kg": ppk, "price_per_l": ppl})

_rx_ppu = re.compile(r"([\d.,]+)\s*€\s*/\s*([^\s]+(?:\s*[^\s]+)?)", re.I)

def _to_float(s):
    try:
        return float(str(s).replace(".", "").replace(",", "."))
    except Exception:
        return None

def _ppu_to_base(ppu_text: str, fallback_unit_price: float) -> float:
    """
    Convierte ppu_text a €/kg o €/l si es posible.
    Si es por unidad/metro/etc., devuelve price_unit (fallback).
    """
    if not isinstance(ppu_text, str) or not ppu_text.strip():
        return float(fallback_unit_price)

    m = _rx_ppu.search(ppu_text.replace("\xa0", " "))
    if not m:
        return float(fallback_unit_price)

    val = _to_float(m.group(1))
    if val is None:
        return float(fallback_unit_price)

    unit = m.group(2).strip().lower()
    unit = unit.replace(" ", "")  # "1 kg" -> "1kg"

    # Normaliza unidades típicas
    if unit in ("kg", "1kg"):
        return float(val)
    if unit in ("l", "1l", "lt", "litro", "litros"):
        return float(val)

    # gramos y mililitros
    if unit in ("g", "1g"):
        return float(val) * 1000.0        # €/g -> €/kg
    if unit in ("100g", "100gr", "100gramos"):
        return float(val) * 10.0          # €/100g -> €/kg
    if unit in ("ml", "1ml"):
        return float(val) * 1000.0        # €/ml -> €/l
    if unit in ("100ml",):
        return float(val) * 10.0          # €/100ml -> €/l

    # por unidad / metro / pack u otras: no se puede pasar a kg/l
    if unit in ("u", "ud", "unidad", "1u", "1ud", "1unidad", "m", "1m"):
        return float(fallback_unit_price)

    # Cualquier otra unidad rara: usa fallback
    return float(fallback_unit_price)

### MERCADONA

In [2]:
df_mercadona = scrape_mercadona(
    start_category_url="https://tienda.mercadona.es/categories/112",
    cp="08203",
    headless=True,
    load_images=True,   # si te importa la columna img_url; pon False para ir aún más rápido
    pause=0.10
)

→ Abriendo https://tienda.mercadona.es/categories/112
✓ CP fijado: 08203
→ Secciones detectadas: 26

=== SECCIÓN 1/26: Aceite, especias y salsas ===
  • Subcategorías: 4
    → 1/4  Aceite, vinagre y sal (id=112)  …click
      ✔ 37 productos en 5.9s
    → 2/4  Especias (id=115)  …click
      ✔ 51 productos en 7.6s
    → 3/4  Mayonesa, ketchup y mostaza (id=116)  …click
      ✔ 19 productos en 3.0s
    → 4/4  Otras salsas (id=117)  …click
      ✔ 41 productos en 5.9s

=== SECCIÓN 2/26: Agua y refrescos ===
  • Subcategorías: 6
    → 1/6  Agua (id=156)  …click
      ✔ 50 productos en 7.4s
    → 2/6  Isotónico y energético (id=163)  …click
      ✔ 39 productos en 5.8s
    → 3/6  Refresco de cola (id=158)  …click
      ✔ 31 productos en 8.1s
    → 4/6  Refresco de naranja y de limón (id=159)  …click
      ✔ 36 productos en 6.5s
    → 5/6  Tónica y bitter (id=161)  …click
      ✔ 11 productos en 2.5s
    → 6/6  Refresco de té y sin gas (id=162)  …click
      ✔ 29 productos en 5.0s

=== SECCI

In [3]:
df_mercadona["store"] = "Mercadona"
df_mercadona["price_per_kg_or_l_or_unit"] = df_mercadona["price_kg"].fillna(df_mercadona["price_l"]).fillna(df_mercadona["price_unit_count"])

In [ ]:
reload_mercadona(df_mercadona)

✅ Mercadona: productos_insertados=4429, categorias_nuevas=152, enlaces_creados=4680


### BONPREU

In [28]:
df_bonpreu = scrape_bonpreu(headless=True)

✅ Cookies aceptadas
✅ Cookies aceptadas
🧮 2119 productos extraídos de https://www.compraonline.bonpreuesclat.cat/categories/frescos/c95cfbf2-501d-433f-bae3-10fcef330b11?sortBy=favorite
✅ Cookies aceptadas
🧮 4172 productos extraídos de https://www.compraonline.bonpreuesclat.cat/categories/alimentaci%C3%B3/c49d1ef2-bf51-44a7-b631-4a35474a21ac?sortBy=favorite
✅ Cookies aceptadas
🧮 1944 productos extraídos de https://www.compraonline.bonpreuesclat.cat/categories/begudes/3660db45-baa3-4c9f-9bb1-7cba443b3c9f?sortBy=favorite
🧮 568 productos extraídos de https://www.compraonline.bonpreuesclat.cat/categories/congelats/79a52e84-e446-47fb-b032-dfa044ecb779?sortBy=favorite
✅ Cookies aceptadas
🧮 819 productos extraídos de https://www.compraonline.bonpreuesclat.cat/categories/l%C3%A0ctics-i-ous/8e6bb6f8-67ac-4a57-8260-c861830774f0?sortBy=favorite
✅ Cookies aceptadas
🧮 2348 productos extraídos de https://www.compraonline.bonpreuesclat.cat/categories/cura-personal/b2c9fc2f-ddb1-40ff-8277-f7b102107f1e?

In [29]:
df_bonpreu["store"] = "Esclat"
df_bonpreu["ppu_num"] = df_bonpreu["price_per_unit_text"].apply(_ppu_simple)

In [30]:
df_up2 = pd.DataFrame({
    "title": df_bonpreu["name"].astype(str).str.strip(),
    "price_unit": pd.to_numeric(df_bonpreu["price"], errors="coerce").fillna(0.0).astype(float).round(4),
    "price_kg": pd.to_numeric(df_bonpreu.get("ppu_num", df_bonpreu["price"]), errors="coerce"),
    "image": df_bonpreu.get("img_url", "").fillna("").astype(str),
    "store": "Esclat",
})
df_up2["price_kg"] = df_up2["price_kg"].fillna(df_up2["price_unit"]).astype(float).round(4)
df_up2 = df_up2.drop_duplicates(subset=["title","store"], keep="first")

# 👉 añade Esclat sin borrar lo de Mercadona
df_up2.to_sql("producto", engine, if_exists="append", index=False)
print(f"OK: {len(df_up2)} filas añadidas en {DB_PATH} para store='Esclat'")

OK: 18317 filas añadidas en baratazo.db para store='Esclat'


### CONSUM

In [31]:
df_consum = scrape_consum(headless=True)
df_consum["name"] = df_consum["brand"]+" "+df_consum["name"]

📂 Categorías detectadas: 13

---- [1/13] https://tienda.consum.es/es/c/bazar/1486?orderById=5&page=1 ----
📄 Páginas detectadas: 10
↕️ Ronda 1: DOM=40
✅ Página 1/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 2/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 3/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 4/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 5/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 6/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 7/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 8/10: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 9/10: DOM=40
↕️ Ronda 1: DOM=8
✅ Página 10/10: DOM=8
🧺 184 productos

---- [2/13] https://tienda.consum.es/es/c/bebidas/1690?orderById=5&page=1 ----
📄 Páginas detectadas: 53
↕️ Ronda 1: DOM=40
✅ Página 1/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 2/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 3/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 4/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 5/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 6/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 7/53: DOM=40
↕️ Ronda 1: DOM=40
✅ Página 8/53: DOM=40
↕️ Ronda 1:

In [33]:
df_up_consum = pd.DataFrame({
    "title": df_consum["name"].astype(str).str.strip(),
    "price_unit": pd.to_numeric(df_consum["price"], errors="coerce").fillna(0.0).astype(float).round(4),
    "image": df_consum.get("image", "").fillna("").astype(str),
    "store": "Consum",
})

ppu_series = df_consum.get("ppu_text", pd.Series([""] * len(df_consum)))
df_up_consum["price_kg"] = [
    round(_ppu_to_base(ppu, fallback_unit_price=pu), 4)
    for ppu, pu in zip(ppu_series.fillna(""), df_up_consum["price_unit"])
]
df_up_consum["price_kg"] = df_up_consum["price_kg"].fillna(df_up_consum["price_unit"]).astype(float).round(4)
df_up_consum = df_up_consum.drop_duplicates(subset=["title","store"], keep="first")

# 👉 añade Consum sin borrar lo de Mercadona
df_up_consum.to_sql("producto", engine, if_exists="append", index=False)
print(f"OK: {len(df_up_consum)} filas añadidas en {DB_PATH} para store='Consum'")

OK: 8942 filas añadidas en baratazo.db para store='Consum'


### TEST SQL

In [52]:
def eur_es(x: float) -> str:
    if pd.isna(x):
        return ""
    s = f"{x:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".") + " €"

with engine.connect() as conn:
    q = text("""
        SELECT title, store, price_unit, price_kg, image
        FROM producto
        WHERE
          (
            LOWER(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(title,
              'á','a'),'é','e'),'í','i'),'ó','o'),'ú','u'),'ü','u')) LIKE '%kiwi%'
            OR
            LOWER(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(title,
              'á','a'),'é','e'),'í','i'),'ó','o'),'ú','u'),'ü','u')) LIKE '%kivi%'
          )
          AND price_kg IS NOT NULL
        ORDER BY price_kg ASC;
    """)
    df_pescado = pd.read_sql(q, conn)

view = df_pescado.copy()
view["Precio unidad"] = view["price_unit"].map(eur_es)
view["Precio kg/L"]   = view["price_kg"].map(eur_es)
view = view[["store", "title", "Precio unidad", "Precio kg/L", "image"]]
view.rename(columns={"store": "Tienda", "title": "Producto"}, inplace=True)
view.index = range(1, len(view) + 1)

pd.set_option("display.max_colwidth", 80)
print(f"🐟 Encontrados: {len(view)} productos (ordenados por kg/L más barato)")
print(view.to_string())

🐟 Encontrados: 18 productos (ordenados por kg/L más barato)
       Tienda                                                                                       Producto Precio unidad Precio kg/L                                                                                                                                       image
1      Esclat                                                                                 Kiwi verd 1 u.        0,59 €      0,59 €  https://www.compraonline.bonpreuesclat.cat/images-v3/dcbcfd72-cf23-44a2-8e14-8a38edd645a3/4fd8664c-1bbb-4dc9-a6aa-4185feeaa33c/300x300.jpg
2      Esclat                                                                   ZESPRI Kiwi groc Zespri 1 u.        0,99 €      0,99 €  https://www.compraonline.bonpreuesclat.cat/images-v3/dcbcfd72-cf23-44a2-8e14-8a38edd645a3/6943d8e5-5ac3-4689-9c90-c710cee6b00b/300x300.jpg
3      Consum                                             TESORO NATUR Refresco Kiwi con Chía Botella 0,45 L       

In [ ]:
df_up1 = pd.DataFrame({
    "title": df["name"].astype(str).str.strip(),
    "price_unit": pd.to_numeric(df["price"], errors="coerce").fillna(0.0).astype(float).round(4),
    "price_kg": pd.to_numeric(df["price_per_kg_or_l"], errors="coerce"),
    "image": df.get("img_url", "").fillna("").astype(str),
    "store": "Mercadona",
})
df_up1["price_kg"] = df_up1["price_kg"].fillna(df_up1["price_unit"]).astype(float).round(4)
df_up1 = df_up1.drop_duplicates(subset=["title","store"], keep="first")


# 👉 crea/arranca la tabla con Mercadona (replace) y deja lista para añadir Esclat
df_up1.to_sql("producto", engine, if_exists="append", index=False)
print(f"OK: {len(df_up1)} filas cargadas en {DB_PATH} para store='Mercadona'")

OK: 4375 filas cargadas en baratazo.db para store='Mercadona'


In [ ]:
#with engine.begin() as conn: conn.exec_driver_sql("DELETE FROM producto")
